In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [33]:
count_m_dict

{Timestamp('2023-01-10 00:00:00'): 0,
 Timestamp('2023-01-15 00:00:00'): 0,
 Timestamp('2023-01-22 00:00:00'): 0,
 Timestamp('2023-01-29 00:00:00'): 0,
 Timestamp('2023-02-05 00:00:00'): 0,
 Timestamp('2023-02-15 00:00:00'): 0,
 Timestamp('2023-02-17 00:00:00'): 0,
 Timestamp('2023-03-01 00:00:00'): 0,
 Timestamp('2023-03-05 00:00:00'): 0,
 Timestamp('2023-03-10 00:00:00'): 0,
 Timestamp('2023-03-20 00:00:00'): 0}

In [71]:
filepath = '../sample_data/sample_data.xlsx'
df = pd.read_excel(filepath)

#standardize x/m count - upper to lowercase
df = df.replace('X', 'x')
df = df.replace('M', 'm')

#get count of Xs and Ms
cols = df.columns
count_x_dict = {c: df.loc[:, c].value_counts()['x'] if 'x' in df.loc[:, c].value_counts() else 0 for c in cols}
count_m_dict = {c: df.loc[:, c].value_counts()['m'] if 'm' in df.loc[:, c].value_counts() else 0 for c in cols}

#replace x by 10, for plotting
df = df.replace('x', '10')
df = df.replace('m', '0')

#cast dataframe to integer
df = df.astype(int)

#get average of group of 6 arrows
round_scores_to_df = defaultdict(lambda: [])
for c in cols:
    vals = df.loc[:, c]
    round_scores_to_df['date'].append(c)
    for i in range(12): 
        round_scores_to_df[i+1].append(np.mean(vals[i: i + 6]))
round_scores_df = pd.DataFrame.from_dict(round_scores_to_df).melt(id_vars=['date'], var_name='ronda', value_name='promedio')
round_scores_avg = round_scores_df[['ronda', 'promedio']].groupby(['ronda']).mean().reset_index()

#get scores by date, long format
scores_long = df.melt(var_name='date', value_name='puntaje')
scores_avg = scores_long.groupby(['date']).mean().reset_index().rename(columns={'puntaje':'promedio'})
scores_total = scores_long.groupby(['date']).sum().reset_index()
scores_all = scores_avg.merge(scores_total)

df

,2023-01-10,2023-01-15,2023-01-22,2023-01-29,2023-02-05,2023-02-15,2023-02-17,2023-03-01,2023-03-05,2023-03-10,2023-03-20
0,3,3,4,6,5,5,10,8,10,10,10
1,6,6,5,7,8,6,9,9,9,8,8
2,7,5,6,6,2,9,7,7,7,10,8
3,6,10,4,7,3,5,10,7,7,9,9
4,5,7,7,6,2,10,10,7,9,10,9
...,...,...,...,...,...,...,...,...,...,...,...
67,6,5,7,9,4,5,10,7,8,9,10
68,4,7,6,9,2,8,7,8,7,10,7
69,6,5,5,6,7,5,9,5,10,9,9
70,0,6,5,5,6,9,7,5,7,10,8


In [3]:
from bokeh.models import BoxAnnotation, LinearAxis, Range1d, Scatter
from bokeh.plotting import figure, show, ColumnDataSource

TOOLS = "pan,wheel_zoom,box_zoom,reset,save, hover"


TOOLTIPS = [
    ("Puntaje promedio", "@promedio"),
    ("Puntaje total", "@puntaje"),
    
]

source = ColumnDataSource(scores_all)

#reduce data size

p = figure(width=1200, height=600, x_axis_type="datetime", tools=TOOLS, y_range=(0,10), tooltips=TOOLTIPS)
p.yaxis.axis_label = "Puntaje Promedio"

# Setting the second y axis range name and range
p.extra_y_ranges = {"puntaje": Range1d(start=0, end=720)}

# Adding the second axis to the plot.  
p.add_layout(LinearAxis(y_range_name="puntaje", axis_label="Puntaje Total"), 'right')

p.line("date", "promedio", source=source, color="#DD1C77", legend_label="puntaje promedio", line_width=2)
#p.circle_x(x=scores_all["date"], y=scores_all["promedio"], size=12,
#              color="#DD1C77", fill_alpha=0.2)
glyph = Scatter(x="date", y="promedio", size=12, fill_color="#DD1C77", marker="circle_x")
p.add_glyph(source, glyph)

#p.circle_x(x=scores_total["date"], y=scores_total["puntaje"],
#         color="#DD1C77", fill_alpha=0.2, y_range_name="puntaje")

yellow = BoxAnnotation(bottom=8, top=10, fill_alpha=0.2, fill_color='#FFE552')
red = BoxAnnotation(bottom=6, top=8, fill_alpha=0.2, fill_color='#F65058')
blue = BoxAnnotation(bottom=4, top=6, fill_alpha=0.2, fill_color='#00B4E4')
black = BoxAnnotation(bottom=2, top=4, fill_alpha=0.2, fill_color='#1b1b1b')

p.add_layout(black)
p.add_layout(blue)
p.add_layout(red)
p.add_layout(yellow)

show(p)

In [41]:
from bokeh.models import BoxAnnotation, LinearAxis, Range1d, Scatter, Band
from bokeh.plotting import figure, show, ColumnDataSource

TOOLS = "pan,wheel_zoom,box_zoom,reset,save, hover"


TOOLTIPS = [
    ("Puntaje promedio", "@promedio"),
    ("Puntaje total", "@puntaje"),
    
]

round_scores_to_plot = round_scores_df[['ronda', 'promedio']].groupby(['ronda']).agg(['sum', 'mean', 'std']).reset_index()
round_scores_to_plot.columns = ['ronda', 'suma', 'promedio', 'std']
round_scores_to_plot['puntaje'] = round_scores_to_plot['promedio'] * 6
round_scores_to_plot['lower'] = round_scores_to_plot['promedio'] - round_scores_to_plot['std']
round_scores_to_plot['upper'] = round_scores_to_plot['promedio'] + round_scores_to_plot['std']

source = ColumnDataSource(round_scores_to_plot)

p = figure(width=1200, height=600, tools=TOOLS, y_range=(0,10), tooltips=TOOLTIPS)
p.yaxis.axis_label = "Puntaje Promedio"

# Setting the second y axis range name and range
p.extra_y_ranges = {"puntaje": Range1d(start=0, end=60)}

# Adding the second axis to the plot.  
p.add_layout(LinearAxis(y_range_name="puntaje", axis_label="Puntaje Total"), 'right')

#p.line("ronda", "promedio", source=source, color="#DD1C77", line_width=2)
band = Band(base="ronda", lower="lower", upper="upper", source=source,
            fill_alpha=0.3, fill_color="gray", line_color="black")
p.add_layout(band)

glyph = Scatter(x="ronda", y="promedio", size=12, fill_color="#DD1C77", marker="circle_x")
p.add_glyph(source, glyph)



yellow = BoxAnnotation(bottom=8, top=10, fill_alpha=0.2, fill_color='#FFE552')
red = BoxAnnotation(bottom=6, top=8, fill_alpha=0.2, fill_color='#F65058')
blue = BoxAnnotation(bottom=4, top=6, fill_alpha=0.2, fill_color='#00B4E4')
black = BoxAnnotation(bottom=2, top=4, fill_alpha=0.2, fill_color='#1b1b1b')

p.add_layout(black)
p.add_layout(blue)
p.add_layout(red)
p.add_layout(yellow)

show(p)

In [67]:
from bokeh.palettes import Paired
rondas = [str(i) for i in range(1,13)]

round_scores_to_plot = round_scores_df.set_index(['date', 'ronda'])['promedio'].unstack().reset_index()
round_scores_to_plot.columns = ['date'] + [str(c) for c in round_scores_to_plot.columns[1:]]
source = ColumnDataSource(round_scores_to_plot)

p = figure(width=1200, height=600, x_axis_type="datetime", tools=TOOLS, y_range=(0,10), tooltips=TOOLTIPS)
p.yaxis.axis_label = "Puntaje Promedio"

# Setting the second y axis range name and range
p.extra_y_ranges = {"puntaje": Range1d(start=0, end=60)}

# Adding the second axis to the plot.  
p.add_layout(LinearAxis(y_range_name="puntaje", axis_label="Puntaje Total"), 'right')

for r, c in zip(rondas, Paired[12]):
    p.line("date", r, source=source, color=c, line_width=2)

    glyph = Scatter(x="date", y=r, size=12, fill_color=c, marker="circle_x")
    p.add_glyph(source, glyph)



yellow = BoxAnnotation(bottom=8, top=10, fill_alpha=0.2, fill_color='#FFE552')
red = BoxAnnotation(bottom=6, top=8, fill_alpha=0.2, fill_color='#F65058')
blue = BoxAnnotation(bottom=4, top=6, fill_alpha=0.2, fill_color='#00B4E4')
black = BoxAnnotation(bottom=2, top=4, fill_alpha=0.2, fill_color='#1b1b1b')

p.add_layout(black)
p.add_layout(blue)
p.add_layout(red)
p.add_layout(yellow)

show(p)

In [76]:
x_m_df = pd.DataFrame.from_dict([{"date": key, "x": val, "m": count_m_dict.get(key)} for key, val in count_x_dict.items()])

In [87]:
from bokeh.models import BoxAnnotation, LinearAxis, Range1d, Scatter, Band
from bokeh.plotting import figure, show, ColumnDataSource

TOOLS = "pan,wheel_zoom,box_zoom,reset,save, hover"


TOOLTIPS = [
    ("Fecha", "@date{%F}"),
    ("Conteo X", "@x"),
    ("Conteo M", "@m"),
    
]

source = ColumnDataSource(x_m_df)

p = figure(width=1200, height=600, x_axis_type="datetime", tools=TOOLS, y_range=(0,10), tooltips=TOOLTIPS, formatters={'@date': 'date'})
p.yaxis.axis_label = "Conteo"

p.line("date", "x", source=source, color="#3CB043", line_width=2, legend_label="X")
glyph = Scatter(x="date", y="x", size=12, fill_color="#03AC13", marker="circle_x")
p.add_glyph(source, glyph)

p.line("date", "m", source=source, color="#DD1C77", line_width=2, legend_label="M")
glyph = Scatter(x="date", y="m", size=12, fill_color="#DD1C77", marker="circle_x")
p.add_glyph(source, glyph)

p.legend.location = "top_left"

show(p)

AttributeError: unexpected attribute 'formatters' to figure, possible attributes are above, active_drag, active_inspect, active_multi, active_scroll, active_tap, align, aspect_ratio, aspect_scale, background_fill_alpha, background_fill_color, below, border_fill_alpha, border_fill_color, center, context_menu, css_classes, disabled, extra_x_ranges, extra_x_scales, extra_y_ranges, extra_y_scales, flow_mode, frame_align, frame_height, frame_width, height, height_policy, hidpi, hold_render, inner_height, inner_width, js_event_callbacks, js_property_callbacks, left, lod_factor, lod_interval, lod_threshold, lod_timeout, margin, match_aspect, max_height, max_width, min_border, min_border_bottom, min_border_left, min_border_right, min_border_top, min_height, min_width, name, outer_height, outer_width, outline_line_alpha, outline_line_cap, outline_line_color, outline_line_dash, outline_line_dash_offset, outline_line_join, outline_line_width, output_backend, renderers, reset_policy, resizable, ri

In [77]:
x_m_df

,date,x,m
0,2023-01-10,0,8
1,2023-01-15,2,5
2,2023-01-22,1,4
3,2023-01-29,3,5
4,2023-02-05,2,3
5,2023-02-15,5,3
6,2023-02-17,4,2
7,2023-03-01,3,2
8,2023-03-05,6,2
9,2023-03-10,8,0
